In [1]:
import boto3
import pandas as pd
from pyathena import connect
import configparser
import time
from io import StringIO

In [2]:
config = configparser.ConfigParser()

In [3]:
config.read('config.ini')

['config.ini']

In [4]:
aws_access_key_id = config.get('aws', 'AWS_ACCESS_KEY')
aws_secret_access_key = config.get('aws', 'AWS_SECRET_KEY')
region = config.get('aws', 'AWS_REGION')
s3_staging_dir = config.get('aws', 'S3_STAGING_AREA')
database = config.get('aws', 'SCHEMA_NAME')
s3_bucket_name = config.get('aws','S3_BUCKET_NAME')
s3_output_dir = 'output'
bucket = config.get('s3','BUCKET_NAME')

In [5]:
athena_client = boto3.client(
    "athena",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region
)

In [6]:
Dict = {}
def download_and_load_query_result(client : boto3.client, query_response : Dict) -> pd.DataFrame:
    while True:
        try:
            # this func will only load the first 1000 rows
            client.get_query_results(
                QueryExecutionId = query_response['QueryExecutionId']
            )
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(0.001)
            else:
                raise err
    
    temp_file_location = "athena_query_result.csv"
    s3_client = boto3.client(
        "s3",
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name=region
    )
    s3_client.download_file(
        s3_bucket_name,
        f"{s3_output_dir}/{query_response['QueryExecutionId']}.csv",
        temp_file_location
    )

    return pd.read_csv(temp_file_location)

In [7]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM customers_dataset",
    QueryExecutionContext = {'Database' : database},
    ResultConfiguration ={
        'OutputLocation' : s3_staging_dir,
        'EncryptionConfiguration' : {'EncryptionOption' : "SSE_S3"}
    }
)

In [8]:
customers_dataset = download_and_load_query_result(athena_client, response)

In [9]:
customers_dataset.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [10]:
customers_dataset.shape

(99441, 5)

In [11]:
customers_dataset.dtypes

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

In [12]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM geolocation",
    QueryExecutionContext = {'Database' : database},
    ResultConfiguration ={
        'OutputLocation' : s3_staging_dir,
        'EncryptionConfiguration' : {'EncryptionOption' : "SSE_S3"}
    }
)

geolocation = download_and_load_query_result(athena_client, response)

In [13]:
geolocation.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,26560,-22.770301,-43.420214,mesquita,RJ
1,26553,-22.793705,-43.430321,mesquita,RJ
2,26562,-22.771253,-43.421624,mesquita,RJ
3,26540,-22.806880,-43.409392,nilopolis,RJ
4,26515,-22.819200,-43.426500,nilopolis,RJ


In [14]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM order_items",
    QueryExecutionContext = {'Database' : database},
    ResultConfiguration ={
        'OutputLocation' : s3_staging_dir,
        'EncryptionConfiguration' : {'EncryptionOption' : "SSE_S3"}
    }
)

order_items = download_and_load_query_result(athena_client, response)

In [15]:
order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [16]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM order_payments",
    QueryExecutionContext = {'Database' : database},
    ResultConfiguration ={
        'OutputLocation' : s3_staging_dir,
        'EncryptionConfiguration' : {'EncryptionOption' : "SSE_S3"}
    }
)

order_payments = download_and_load_query_result(athena_client, response)

In [17]:
order_payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [18]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM orders",
    QueryExecutionContext = {'Database' : database},
    ResultConfiguration ={
        'OutputLocation' : s3_staging_dir,
        'EncryptionConfiguration' : {'EncryptionOption' : "SSE_S3"}
    }
)

orders = download_and_load_query_result(athena_client, response)

In [19]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [20]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM product_category_name_translation",
    QueryExecutionContext = {'Database' : database},
    ResultConfiguration ={
        'OutputLocation' : s3_staging_dir,
        'EncryptionConfiguration' : {'EncryptionOption' : "SSE_S3"}
    }
)

product_category_name_translation = download_and_load_query_result(athena_client, response)

In [21]:
product_category_name_translation.head()

,product_category_name,product_category_name_english
0,product_category_name,product_category_name_english
1,beleza_saude,health_beauty
2,informatica_acessorios,computers_accessories
3,automotivo,auto
4,cama_mesa_banho,bed_bath_table


In [22]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM products",
    QueryExecutionContext = {'Database' : database},
    ResultConfiguration ={
        'OutputLocation' : s3_staging_dir,
        'EncryptionConfiguration' : {'EncryptionOption' : "SSE_S3"}
    }
)

products = download_and_load_query_result(athena_client, response)

In [23]:
products.head()

,product_id,product_category_name,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [24]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM reviews",
    QueryExecutionContext = {'Database' : database},
    ResultConfiguration ={
        'OutputLocation' : s3_staging_dir,
        'EncryptionConfiguration' : {'EncryptionOption' : "SSE_S3"}
    }
)

reviews = download_and_load_query_result(athena_client, response)

In [25]:
reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [26]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM reviews_translated",
    QueryExecutionContext = {'Database' : database},
    ResultConfiguration ={
        'OutputLocation' : s3_staging_dir,
        'EncryptionConfiguration' : {'EncryptionOption' : "SSE_S3"}
    }
)

reviews_translated = download_and_load_query_result(athena_client, response)

In [27]:
reviews_translated.head()

,unnamed,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,I received it well before the stipulated time.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Congratulations Lannister stores I loved shopp...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [28]:
response = athena_client.start_query_execution(
    QueryString = "SELECT * FROM sellers",
    QueryExecutionContext = {'Database' : database},
    ResultConfiguration ={
        'OutputLocation' : s3_staging_dir,
        'EncryptionConfiguration' : {'EncryptionOption' : "SSE_S3"}
    }
)

sellers = download_and_load_query_result(athena_client, response)

In [29]:
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


## Transformation

In [30]:
reviews_translated = reviews_translated.drop(columns=['unnamed'])

In [31]:
reviews_translated.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,I received it well before the stipulated time.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Congratulations Lannister stores I loved shopp...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [32]:
reviews_translated.dtypes

review_id                  object
order_id                   object
review_score               object
review_comment_title       object
review_comment_message     object
review_creation_date       object
review_answer_timestamp    object
dtype: object

In [33]:
reviews_translated['review_creation_date'] = pd.to_datetime(reviews_translated['review_creation_date'])

In [34]:
reviews_translated['review_answer_timestamp'] = pd.to_datetime(reviews_translated['review_answer_timestamp'])

In [35]:
reviews_translated.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,I received it well before the stipulated time.,2017-04-21,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Congratulations Lannister stores I loved shopp...,2018-03-01,2018-03-02 10:26:53


In [36]:
reviews_translated.dtypes

review_id                          object
order_id                           object
review_score                       object
review_comment_title               object
review_comment_message             object
review_creation_date       datetime64[ns]
review_answer_timestamp    datetime64[ns]
dtype: object

In [37]:
geolocation.dtypes

geolocation_zip_code_prefix      int64
geolocation_lat                float64
geolocation_lng                float64
geolocation_city                object
geolocation_state               object
dtype: object

In [38]:
fact_orders_info = pd.DataFrame()

In [39]:
fact_orders_info['order_id'] = orders['order_id']

In [40]:
fact_orders_info.head()

,order_id
0,e481f51cbdc54678b7cc49136f2d6af7
1,53cdb2fc8bc7dce0b6741e2150273451
2,47770eb9100c2d0c44946d9cf07ec65d
3,949d5b44dbf5de918fe9c16f97b45f8a
4,ad21c59c0840e6cb83a9ceb5573f8159


In [41]:
fact_orders_info = pd.merge(order_items,orders[['order_id','customer_id']], on='order_id',how='left')

In [42]:
fact_orders_info.shape

(112650, 8)

In [43]:
fact_orders_info.tail(5)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,b51593916b4b8e0d6f66f2ae24f2673d
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,84c5d4fbaf120aae381fad077416eaa0
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,29309aa813182aaddc9b259e31b870e6
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,b5e6afd5a41800fdf401e0272ca74655
112649,fffe41c64501cc87c801fd61db3f6244,1,350688d9dc1e75ff97be326363655e01,f7ccf836d21b2fb1de37564105216cc1,2018-06-12 17:10:13,43.00,12.79,96d649da0cc4ff33bb408b199d4c7dcf


In [44]:
fact_orders_info['product_id'].isnull().sum()

0

In [45]:
fact_orders_info = pd.merge(fact_orders_info,order_payments, on='order_id',how = 'left')

In [46]:
fact_orders_info.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,payment_sequential,payment_type,payment_installments,payment_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,1.0,credit_card,2.0,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,1.0,credit_card,3.0,259.83
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,1.0,credit_card,5.0,216.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,1.0,credit_card,2.0,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,1.0,credit_card,3.0,218.04


In [47]:
fact_orders_info = pd.merge(fact_orders_info,reviews_translated[['order_id','review_id','review_score']], on = 'order_id', how= 'left')

In [48]:
fact_orders_info.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,1.0,credit_card,2.0,72.19,97ca439bc427b48bc1cd7177abe71365,5
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,1.0,credit_card,3.0,259.83,7b07bacd811c4117b742569b04ce3580,4
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,1.0,credit_card,5.0,216.87,0c5b33dea94867d1ac402749e5438e8b,5
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,1.0,credit_card,2.0,25.78,f4028d019cb58564807486a6aaf33817,4
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,1.0,credit_card,3.0,218.04,940144190dcba6351888cafa43f3a3a5,5


In [49]:
fact_orders_info.shape

(118310, 14)

In [50]:
product_category_name_translation.head()

,product_category_name,product_category_name_english
0,product_category_name,product_category_name_english
1,beleza_saude,health_beauty
2,informatica_acessorios,computers_accessories
3,automotivo,auto
4,cama_mesa_banho,bed_bath_table


In [51]:
new_header = product_category_name_translation.iloc[0]

In [52]:
new_header

product_category_name                    product_category_name
product_category_name_english    product_category_name_english
Name: 0, dtype: object

In [53]:
product_category_name_translation = product_category_name_translation[1:]

In [54]:
product_category_name_translation.head()

,product_category_name,product_category_name_english
1,beleza_saude,health_beauty
2,informatica_acessorios,computers_accessories
3,automotivo,auto
4,cama_mesa_banho,bed_bath_table
5,moveis_decoracao,furniture_decor


In [55]:
product_category_name_translation.columns = new_header

In [56]:
product_category_name_translation.head()

,product_category_name,product_category_name_english
1,beleza_saude,health_beauty
2,informatica_acessorios,computers_accessories
3,automotivo,auto
4,cama_mesa_banho,bed_bath_table
5,moveis_decoracao,furniture_decor


## Dimension tables

In [60]:
dim_geolocation = pd.DataFrame()

In [61]:
dim_geolocation = geolocation.copy(deep=True)

In [62]:
dim_geolocation.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,26560,-22.770301,-43.420214,mesquita,RJ
1,26553,-22.793705,-43.430321,mesquita,RJ
2,26562,-22.771253,-43.421624,mesquita,RJ
3,26540,-22.806880,-43.409392,nilopolis,RJ
4,26515,-22.819200,-43.426500,nilopolis,RJ


In [63]:
dim_customers = pd.DataFrame()

In [64]:
dim_customers = customers_dataset.copy(deep=True)

In [65]:
dim_customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [66]:
dim_sellers = pd.DataFrame()

In [67]:
dim_sellers = sellers.copy(deep=True)

In [68]:
dim_sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [69]:
dim_products = pd.DataFrame()

In [70]:
dim_products = products.copy(deep=True)

In [71]:
dim_products.head()

,product_id,product_category_name,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [72]:
dim_products = pd.merge(dim_products,product_category_name_translation[['product_category_name','product_category_name_english']],on='product_category_name',how='outer')

In [73]:
dim_products.head()

,product_id,product_category_name,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,6a2fb4dd53d2cdb88e0432f1284a004c,perfumaria,39.0,346.0,2.0,400.0,27.0,5.0,20.0,perfumery
2,0d009643171aee696f4733340bc2fdd0,perfumaria,52.0,150.0,1.0,422.0,21.0,16.0,18.0,perfumery
3,b1eae565a61935e0011ee7682fef9dc9,perfumaria,49.0,460.0,2.0,267.0,17.0,13.0,17.0,perfumery
4,8da90b37f0fb171b4877c124f965b1f6,perfumaria,56.0,733.0,3.0,377.0,18.0,13.0,15.0,perfumery


In [74]:
dim_products = dim_products[['product_id','product_category_name','product_category_name_english','product_name_length','product_description_length','product_photos_qty','product_weight_g','product_length_cm','product_height_cm','product_width_cm']]

In [75]:
dim_products.head()

,product_id,product_category_name,product_category_name_english,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,perfumery,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,6a2fb4dd53d2cdb88e0432f1284a004c,perfumaria,perfumery,39.0,346.0,2.0,400.0,27.0,5.0,20.0
2,0d009643171aee696f4733340bc2fdd0,perfumaria,perfumery,52.0,150.0,1.0,422.0,21.0,16.0,18.0
3,b1eae565a61935e0011ee7682fef9dc9,perfumaria,perfumery,49.0,460.0,2.0,267.0,17.0,13.0,17.0
4,8da90b37f0fb171b4877c124f965b1f6,perfumaria,perfumery,56.0,733.0,3.0,377.0,18.0,13.0,15.0


In [76]:
dim_products.shape

(32951, 10)

In [77]:
dim_reviews = pd.DataFrame()

In [78]:
dim_reviews = reviews_translated.copy(deep=True)

In [79]:
dim_reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,I received it well before the stipulated time.,2017-04-21,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Congratulations Lannister stores I loved shopp...,2018-03-01,2018-03-02 10:26:53


In [80]:
dim_reviews = dim_reviews.drop(columns=['review_score'])

In [81]:
dim_reviews.dtypes

review_id                          object
order_id                           object
review_comment_title               object
review_comment_message             object
review_creation_date       datetime64[ns]
review_answer_timestamp    datetime64[ns]
dtype: object

In [82]:
orders_date_dim = pd.DataFrame()

In [83]:
orders_date_dim = orders.copy(deep=True)

In [84]:
orders_date_dim.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [85]:
orders_date_dim['order_purchase_timestamp'] = pd.to_datetime(orders_date_dim['order_purchase_timestamp'], errors='coerce')
orders_date_dim['order_approved_at'] = pd.to_datetime(orders_date_dim['order_approved_at'], errors='coerce')
orders_date_dim['order_delivered_carrier_date'] = pd.to_datetime(orders_date_dim['order_delivered_carrier_date'], errors='coerce')
orders_date_dim['order_delivered_customer_date'] = pd.to_datetime(orders_date_dim['order_delivered_customer_date'], errors='coerce')
orders_date_dim['order_estimated_delivery_date'] = pd.to_datetime(orders_date_dim['order_estimated_delivery_date'], errors='coerce')

In [86]:
dates = pd.concat([
    orders_date_dim['order_purchase_timestamp'].dropna(),
    orders_date_dim['order_approved_at'].dropna(),
    orders_date_dim['order_delivered_carrier_date'].dropna(),
    orders_date_dim['order_delivered_customer_date'].dropna(),
    orders_date_dim['order_estimated_delivery_date'].dropna()
]).drop_duplicates().reset_index(drop=True)

In [87]:
date_dim = pd.DataFrame({
    'date': dates,
    'date_id': range(1, len(dates) + 1)
})

In [88]:
date_dim['date'] = pd.to_datetime(date_dim['date'])


In [89]:
date_dim['day'] = date_dim['date'].dt.day
date_dim['month'] = date_dim['date'].dt.month
date_dim['year'] = date_dim['date'].dt.year
date_dim['quarter'] = date_dim['date'].dt.quarter
date_dim['week'] = date_dim['date'].dt.isocalendar().week
date_dim['day_of_week'] = date_dim['date'].dt.day_name()

In [90]:
orders_date_dim = orders_date_dim.merge(date_dim[['date', 'date_id']], how='left', left_on='order_purchase_timestamp', right_on='date')
orders_date_dim = orders_date_dim.rename(columns={'date_id': 'order_purchase_timestamp_key'})
orders_date_dim = orders_date_dim.drop(columns=['date'])

orders_date_dim = orders_date_dim.merge(date_dim[['date', 'date_id']], how='left', left_on='order_approved_at', right_on='date')
orders_date_dim = orders_date_dim.rename(columns={'date_id': 'order_approved_at_key'})
orders_date_dim['order_approved_at_key'] = orders_date_dim['order_approved_at_key'].fillna(-1).astype(int)
orders_date_dim = orders_date_dim.drop(columns=['date'])

orders_date_dim = orders_date_dim.merge(date_dim[['date', 'date_id']], how='left', left_on='order_delivered_carrier_date', right_on='date')
orders_date_dim = orders_date_dim.rename(columns={'date_id': 'order_delivered_carrier_date_key'})
orders_date_dim['order_delivered_carrier_date_key'] = orders_date_dim['order_delivered_carrier_date_key'].fillna(-1).astype(int)
orders_date_dim = orders_date_dim.drop(columns=['date'])

orders_date_dim = orders_date_dim.merge(date_dim[['date', 'date_id']], how='left', left_on='order_delivered_customer_date', right_on='date')
orders_date_dim = orders_date_dim.rename(columns={'date_id': 'order_delivered_customer_date_key'})
orders_date_dim['order_delivered_customer_date_key'] = orders_date_dim['order_delivered_customer_date_key'].fillna(-1).astype(int)
orders_date_dim = orders_date_dim.drop(columns=['date'])

orders_date_dim = orders_date_dim.merge(date_dim[['date', 'date_id']], how='left', left_on='order_estimated_delivery_date', right_on='date')
orders_date_dim = orders_date_dim.rename(columns={'date_id': 'order_estimated_delivery_date_key'})
orders_date_dim['order_estimated_delivery_date_key'] = orders_date_dim['order_estimated_delivery_date_key'].fillna(-1).astype(int)
orders_date_dim = orders_date_dim.drop(columns=['date'])

In [91]:
orders_date_dim.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_purchase_timestamp_key,order_approved_at_key,order_delivered_carrier_date_key,order_delivered_customer_date_key,order_estimated_delivery_date_key
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1,98876,188052,268625,362872
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,2,98877,188053,268626,362873
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,3,98878,188054,268627,362874
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,4,98879,188055,268628,362875
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,5,98880,188056,268629,362876


In [92]:
orders_date_dim.dtypes

order_id                                     object
customer_id                                  object
order_status                                 object
order_purchase_timestamp             datetime64[ns]
order_approved_at                    datetime64[ns]
order_delivered_carrier_date         datetime64[ns]
order_delivered_customer_date        datetime64[ns]
order_estimated_delivery_date        datetime64[ns]
order_purchase_timestamp_key                  int64
order_approved_at_key                         int32
order_delivered_carrier_date_key              int32
order_delivered_customer_date_key             int32
order_estimated_delivery_date_key             int32
dtype: object

In [93]:
orders_date_dim = orders_date_dim.drop(columns=['order_purchase_timestamp', 'order_approved_at', 
                                                     'order_delivered_carrier_date', 'order_delivered_customer_date', 
                                                     'order_estimated_delivery_date'])


In [94]:
orders_date_dim.head()

,order_id,customer_id,order_status,order_purchase_timestamp_key,order_approved_at_key,order_delivered_carrier_date_key,order_delivered_customer_date_key,order_estimated_delivery_date_key
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,1,98876,188052,268625,362872
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2,98877,188053,268626,362873
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,3,98878,188054,268627,362874
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,4,98879,188055,268628,362875
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,5,98880,188056,268629,362876


In [95]:
date_dim.head()

,date,date_id,day,month,year,quarter,week,day_of_week
0,2017-10-02 10:56:33,1,2,10,2017,4,40,Monday
1,2018-07-24 20:41:37,2,24,7,2018,3,30,Tuesday
2,2018-08-08 08:38:49,3,8,8,2018,3,32,Wednesday
3,2017-11-18 19:28:06,4,18,11,2017,4,46,Saturday
4,2018-02-13 21:18:39,5,13,2,2018,1,7,Tuesday


In [96]:
dim_geolocation.isnull().sum()

geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64

# Storing data to S3

In [183]:
session = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region
)

In [96]:
bucket

'sakshi-ecommerce-de-project'

In [97]:
s3_resource = boto3.resource(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region
)

In [189]:
csv_buffer1 = StringIO()
dim_geolocation.to_csv(csv_buffer1, index=False)
s3_resource.Object(bucket, 'files/output/dim_geolocation.csv').put(Body=csv_buffer1.getvalue())

{'ResponseMetadata': {'RequestId': '785WMZG50QCWBC9A',
  'HostId': 'sePvVliPUpyiwY2oU6RLNHtvR8xm8liXXuE0qCRiCdXdhL3DGJrtO0L7A3sZzd4kcFSackeDR7iCU8HN4pnQ5Q==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'sePvVliPUpyiwY2oU6RLNHtvR8xm8liXXuE0qCRiCdXdhL3DGJrtO0L7A3sZzd4kcFSackeDR7iCU8HN4pnQ5Q==',
   'x-amz-request-id': '785WMZG50QCWBC9A',
   'date': 'Mon, 26 Aug 2024 09:08:19 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"3309ca02fb3a1e800ea0161def741ccd"',
   'server': 'AmazonS3',
   'content-length': '0',
   'connection': 'close'},
  'RetryAttempts': 0},
 'ETag': '"3309ca02fb3a1e800ea0161def741ccd"',
 'ServerSideEncryption': 'AES256'}

In [190]:
csv_buffer2 = StringIO()
dim_customers.to_csv(csv_buffer2, index=False)
s3_resource.Object(bucket, 'files/output/dim_customers.csv').put(Body=csv_buffer2.getvalue())

{'ResponseMetadata': {'RequestId': 'S8DCPS7XZFQ53PFK',
  'HostId': 'vftcwH58fz+jezbakO4TEnaH08gWJNskFgzt2PJJ3xnpNdOq65EphQsnk2dJTLh3TYGgoGr0KpY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'vftcwH58fz+jezbakO4TEnaH08gWJNskFgzt2PJJ3xnpNdOq65EphQsnk2dJTLh3TYGgoGr0KpY=',
   'x-amz-request-id': 'S8DCPS7XZFQ53PFK',
   'date': 'Mon, 26 Aug 2024 09:20:08 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"9c7b9a40546f5020ce008ea5e687882a"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"9c7b9a40546f5020ce008ea5e687882a"',
 'ServerSideEncryption': 'AES256'}

In [191]:
csv_buffer3 = StringIO()
dim_sellers.to_csv(csv_buffer3, index=False)
s3_resource.Object(bucket, 'files/output/dim_sellers.csv').put(Body=csv_buffer3.getvalue())

{'ResponseMetadata': {'RequestId': 'KS3VV9R64XPE1QD3',
  'HostId': 'VdiDC3qB14PKp8nmpP49MdyF85W3UtAGi5u5gSfHjFgAGzX/gYgjEQXfFH6tY+vyh6LvfPXLKxM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'VdiDC3qB14PKp8nmpP49MdyF85W3UtAGi5u5gSfHjFgAGzX/gYgjEQXfFH6tY+vyh6LvfPXLKxM=',
   'x-amz-request-id': 'KS3VV9R64XPE1QD3',
   'date': 'Mon, 26 Aug 2024 09:26:00 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"3d406935dc9b3761cbe083ac9e3a2af6"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"3d406935dc9b3761cbe083ac9e3a2af6"',
 'ServerSideEncryption': 'AES256'}

In [192]:
csv_buffer4 = StringIO()
fact_orders_info.to_csv(csv_buffer4, index=False)
s3_resource.Object(bucket, 'files/output/fact_orders_info.csv').put(Body=csv_buffer4.getvalue())

{'ResponseMetadata': {'RequestId': 'VXSN18ZS9V0QANCJ',
  'HostId': 'YhVkCCsIq8eLiijYJN1itzu7LRwIH/JlEOELdEyFzLZ76KMxc3eENsSBqEAtfEhDeO8mrO4/aQpjRhcONVcpCB/23jJiGKTC',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'YhVkCCsIq8eLiijYJN1itzu7LRwIH/JlEOELdEyFzLZ76KMxc3eENsSBqEAtfEhDeO8mrO4/aQpjRhcONVcpCB/23jJiGKTC',
   'x-amz-request-id': 'VXSN18ZS9V0QANCJ',
   'date': 'Mon, 26 Aug 2024 09:27:07 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"e8bb5348b656536242ba1c202d0312bc"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"e8bb5348b656536242ba1c202d0312bc"',
 'ServerSideEncryption': 'AES256'}

In [193]:
csv_buffer5 = StringIO()
dim_reviews.to_csv(csv_buffer5, index=False)
s3_resource.Object(bucket, 'files/output/dim_reviews.csv').put(Body=csv_buffer5.getvalue())

{'ResponseMetadata': {'RequestId': 'QF8NTRRAK3EE7CD3',
  'HostId': 's2tBh0ucElPUXDOqnB2qNaMk+Ho0I3ayp+QM3bRUr3MAcQa7/3Nt9z2DXIsrtxZsuSzzJh4w3+I=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 's2tBh0ucElPUXDOqnB2qNaMk+Ho0I3ayp+QM3bRUr3MAcQa7/3Nt9z2DXIsrtxZsuSzzJh4w3+I=',
   'x-amz-request-id': 'QF8NTRRAK3EE7CD3',
   'date': 'Mon, 26 Aug 2024 09:32:46 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"a0f5613dcdcfb84f4862553bc1c79756"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"a0f5613dcdcfb84f4862553bc1c79756"',
 'ServerSideEncryption': 'AES256'}

In [194]:
csv_buffer6 = StringIO()
dim_products.to_csv(csv_buffer6, index=False)
s3_resource.Object(bucket, 'files/output/dim_products.csv').put(Body=csv_buffer6.getvalue())

{'ResponseMetadata': {'RequestId': 'FESZZ3VKTE3QCA97',
  'HostId': 'O5KymLBYnGaH9BbkjvkvIf7QyfYARs8q0JgjE/xKnsijXlikwlN2Z4sPJwKXH9v1UaDFRwn1ins=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'O5KymLBYnGaH9BbkjvkvIf7QyfYARs8q0JgjE/xKnsijXlikwlN2Z4sPJwKXH9v1UaDFRwn1ins=',
   'x-amz-request-id': 'FESZZ3VKTE3QCA97',
   'date': 'Mon, 26 Aug 2024 09:35:11 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"4561b0aa25e36b3e779b5659921e9b97"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"4561b0aa25e36b3e779b5659921e9b97"',
 'ServerSideEncryption': 'AES256'}

In [195]:
csv_buffer7 = StringIO()
orders_date_dim.to_csv(csv_buffer7, index=False)
s3_resource.Object(bucket, 'files/output/orders_date_dim.csv').put(Body=csv_buffer7.getvalue())

{'ResponseMetadata': {'RequestId': 'XF0YQ81ZZFN610Q9',
  'HostId': 'DGO7FiJGDfw28JXmx3b76Ukwy4XnamK4WIGpMaNFbHV6m+ZluwYoLUAQpyVqxTwulCe72siBTgpZ5fUbhnB2cg==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'DGO7FiJGDfw28JXmx3b76Ukwy4XnamK4WIGpMaNFbHV6m+ZluwYoLUAQpyVqxTwulCe72siBTgpZ5fUbhnB2cg==',
   'x-amz-request-id': 'XF0YQ81ZZFN610Q9',
   'date': 'Mon, 26 Aug 2024 09:35:45 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"199d45016827ca30221a7660a85a0678"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"199d45016827ca30221a7660a85a0678"',
 'ServerSideEncryption': 'AES256'}

In [196]:
csv_buffer8 = StringIO()
date_dim.to_csv(csv_buffer8, index=False)
s3_resource.Object(bucket, 'files/output/date_dim.csv').put(Body=csv_buffer8.getvalue())

{'ResponseMetadata': {'RequestId': '9V2E8BYX2V8KTQ4D',
  'HostId': 'UbvnnKvFOzX/qGqsw1upbCLB+L4TOybPqCbjSt7NLv8UYHFbSd0wvBxZGIaYhrGpCH91d6IEzDY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'UbvnnKvFOzX/qGqsw1upbCLB+L4TOybPqCbjSt7NLv8UYHFbSd0wvBxZGIaYhrGpCH91d6IEzDY=',
   'x-amz-request-id': '9V2E8BYX2V8KTQ4D',
   'date': 'Mon, 26 Aug 2024 09:37:06 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"c375c548be6423e4296a1226034a8c62"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"c375c548be6423e4296a1226034a8c62"',
 'ServerSideEncryption': 'AES256'}

# Getting the schema of DataFrames to create tables in Redshift

In [98]:
dim_geolocation_sql = pd.io.sql.get_schema(dim_geolocation.reset_index(),'dim_geolocation')
print(''.join(dim_geolocation_sql))

CREATE TABLE "dim_geolocation" (
"index" INTEGER,
  "geolocation_zip_code_prefix" INTEGER,
  "geolocation_lat" REAL,
  "geolocation_lng" REAL,
  "geolocation_city" TEXT,
  "geolocation_state" TEXT
)


In [99]:
dim_customers_sql = pd.io.sql.get_schema(dim_customers.reset_index(),'dim_customers')
print(''.join(dim_customers_sql))

CREATE TABLE "dim_customers" (
"index" INTEGER,
  "customer_id" TEXT,
  "customer_unique_id" TEXT,
  "customer_zip_code_prefix" INTEGER,
  "customer_city" TEXT,
  "customer_state" TEXT
)


In [100]:
dim_sellers_sql = pd.io.sql.get_schema(dim_sellers.reset_index(),'dim_sellers')
print(''.join(dim_sellers_sql))

CREATE TABLE "dim_sellers" (
"index" INTEGER,
  "seller_id" TEXT,
  "seller_zip_code_prefix" INTEGER,
  "seller_city" TEXT,
  "seller_state" TEXT
)


In [101]:
fact_orders_info_sql = pd.io.sql.get_schema(fact_orders_info.reset_index(),'fact_orders_info')
print(''.join(fact_orders_info_sql))

CREATE TABLE "fact_orders_info" (
"index" INTEGER,
  "order_id" TEXT,
  "order_item_id" INTEGER,
  "product_id" TEXT,
  "seller_id" TEXT,
  "shipping_limit_date" TEXT,
  "price" REAL,
  "freight_value" REAL,
  "customer_id" TEXT,
  "payment_sequential" REAL,
  "payment_type" TEXT,
  "payment_installments" REAL,
  "payment_value" REAL,
  "review_id" TEXT,
  "review_score" TEXT
)


In [102]:
dim_products_sql = pd.io.sql.get_schema(dim_products.reset_index(),'dim_products')
print(''.join(dim_products_sql))

CREATE TABLE "dim_products" (
"index" INTEGER,
  "product_id" TEXT,
  "product_category_name" TEXT,
  "product_category_name_english" TEXT,
  "product_name_length" REAL,
  "product_description_length" REAL,
  "product_photos_qty" REAL,
  "product_weight_g" REAL,
  "product_length_cm" REAL,
  "product_height_cm" REAL,
  "product_width_cm" REAL
)


In [103]:
orders_date_dim_sql = pd.io.sql.get_schema(orders_date_dim.reset_index(),'orders_date_dim')
print(''.join(orders_date_dim_sql))

CREATE TABLE "orders_date_dim" (
"index" INTEGER,
  "order_id" TEXT,
  "customer_id" TEXT,
  "order_status" TEXT,
  "order_purchase_timestamp_key" INTEGER,
  "order_approved_at_key" INTEGER,
  "order_delivered_carrier_date_key" INTEGER,
  "order_delivered_customer_date_key" INTEGER,
  "order_estimated_delivery_date_key" INTEGER
)


In [104]:
date_dim_sql = pd.io.sql.get_schema(date_dim.reset_index(),'date_dim')
print(''.join(date_dim_sql))

CREATE TABLE "date_dim" (
"index" INTEGER,
  "date" TIMESTAMP,
  "date_id" INTEGER,
  "day" INTEGER,
  "month" INTEGER,
  "year" INTEGER,
  "quarter" INTEGER,
  "week" INTEGER,
  "day_of_week" TEXT
)


In [105]:
dim_reviews_sql = pd.io.sql.get_schema(dim_reviews.reset_index(),'dim_reviews')
print(''.join(dim_reviews_sql))

CREATE TABLE "dim_reviews" (
"index" INTEGER,
  "review_id" TEXT,
  "order_id" TEXT,
  "review_comment_title" TEXT,
  "review_comment_message" TEXT,
  "review_creation_date" TIMESTAMP,
  "review_answer_timestamp" TIMESTAMP
)


In [106]:
dim_products.dtypes

product_id                        object
product_category_name             object
product_category_name_english     object
product_name_length              float64
product_description_length       float64
product_photos_qty               float64
product_weight_g                 float64
product_length_cm                float64
product_height_cm                float64
product_width_cm                 float64
dtype: object